In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import xlrd 
import json


# # Import API key
# from api_keys import api_key


In [2]:
#import data from url to get the state abbr
url="http://worldpopulationreview.com/static/states/name-abbr.json"
d = requests.get(url).json()

states_abbr = pd.DataFrame(d.items() , columns=["State","State Abbr"])

states_abbr.head()

,State,State Abbr
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [3]:
#import excel file with obesity, unemployment, education
xls = pd.ExcelFile('Resources/2016 County Health Rankings Data - v3.xls')
df = pd.read_excel(xls, 'Ranked Measure Data', skiprows=1)


In [4]:
df.head()


,FIPS,State,County,# Deaths,Years of Potential Life Lost Rate,95% CI - Low,95% CI - High,Quartile,% Fair/Poor,95% CI - Low.1,...,# Workers,% Drive Alone,95% CI - Low.20,95% CI - High.20,Quartile.33,# Workers who Drive Alone,% Long Commute - Drives Alone,95% CI - Low.21,95% CI - High.21,Quartile.34
0,1001.0,Alabama,Autauga,809.0,9215.1,8307.8,10122.3,2,19.4,18.8,...,24290.0,89.230136,87.152547,91.307725,4,21674.0,41.7,37.996787,45.403213,3
1,1003.0,Alabama,Baldwin,2506.0,7454.6,7001.9,7907.3,1,16.0,15.6,...,82134.0,84.071152,82.543906,85.598398,2,69051.0,36.8,34.609473,38.990527,3
2,1005.0,Alabama,Barbour,410.0,8499.9,7256.9,9742.8,1,25.7,25.0,...,9108.0,83.519982,81.092529,85.947436,2,7607.0,35.1,29.689536,40.510464,2
3,1007.0,Alabama,Bibb,414.0,11007.7,9346.3,12669.1,3,22.0,21.3,...,7773.0,84.034478,80.071760,87.997196,2,6532.0,45.7,38.399757,53.000243,4
4,1009.0,Alabama,Blount,876.0,8655.9,7799.8,9512.0,1,20.7,20.0,...,22168.0,85.073078,82.839898,87.306258,3,18859.0,59.0,54.229577,63.770423,4


In [5]:
#rename columns used for analysis
df.rename(columns={ df.columns[108]: "Unemployment Percent" }, inplace = True)
df.rename(columns={ df.columns[31]: "Obesity Percent" }, inplace = True)
df.rename(columns={ df.columns[102]: "Some College Percent" }, inplace = True)
df.rename(columns={ df.columns[98]: "High School Grad Percent" }, inplace = True)


#merge data
stats_df = pd.merge(states_abbr, df,
                                 how='left', on='State')

stats_df.head()


,State,State Abbr,FIPS,County,# Deaths,Years of Potential Life Lost Rate,95% CI - Low,95% CI - High,Quartile,% Fair/Poor,...,# Workers,% Drive Alone,95% CI - Low.20,95% CI - High.20,Quartile.33,# Workers who Drive Alone,% Long Commute - Drives Alone,95% CI - Low.21,95% CI - High.21,Quartile.34
0,Alabama,AL,1001.0,Autauga,809.0,9215.1,8307.8,10122.3,2,19.4,...,24290.0,89.230136,87.152547,91.307725,4,21674.0,41.7,37.996787,45.403213,3
1,Alabama,AL,1003.0,Baldwin,2506.0,7454.6,7001.9,7907.3,1,16.0,...,82134.0,84.071152,82.543906,85.598398,2,69051.0,36.8,34.609473,38.990527,3
2,Alabama,AL,1005.0,Barbour,410.0,8499.9,7256.9,9742.8,1,25.7,...,9108.0,83.519982,81.092529,85.947436,2,7607.0,35.1,29.689536,40.510464,2
3,Alabama,AL,1007.0,Bibb,414.0,11007.7,9346.3,12669.1,3,22.0,...,7773.0,84.034478,80.071760,87.997196,2,6532.0,45.7,38.399757,53.000243,4
4,Alabama,AL,1009.0,Blount,876.0,8655.9,7799.8,9512.0,1,20.7,...,22168.0,85.073078,82.839898,87.306258,3,18859.0,59.0,54.229577,63.770423,4


In [6]:
#clean up data to show only stats in comparision
Unemployment_Percent=stats_df["Unemployment Percent"]
Obesity_Percent=stats_df["Obesity Percent"]
Some_College=stats_df["Some College Percent"]
High_School=stats_df["High School Grad Percent"]
State_Name=stats_df["State"]
County=stats_df["County"]
state_abbr=stats_df["State Abbr"]

#create new Dataframe with just information needed
formatted_df= pd.DataFrame({"State": State_Name,
                        "State Abbr": state_abbr,
                        "County": County,
                        "Unemployment (%)": Unemployment_Percent,
                        "Obesity (%)": Obesity_Percent,
                        "Some College (%)": Some_College,
                        "High School Graduate (%)": High_School
                       })

formatted_df.head()

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%)
0,Alabama,AL,Autauga,5.883047,30.9,56.470750,83.000000
1,Alabama,AL,Baldwin,6.053720,26.7,62.335918,80.000000
2,Alabama,AL,Barbour,10.809275,40.8,43.358568,80.530303
3,Alabama,AL,Bibb,7.137850,40.1,48.927875,81.000000
4,Alabama,AL,Blount,6.146908,32.4,49.149934,84.975362


In [7]:
income_xl = pd.ExcelFile('Resources/county income level PROJECT.xlsx')
income_df = pd.read_excel(income_xl, skiprows = [1,2,3,4,5,6,])
income_df.head()

,Unnamed: 0,"Table 1. Per Capita Personal Income by County, 2015 - 2017",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Alabama,Autauga,38591.0,39509.0,40484.0,9,2.4,2.5,67
1,Alabama,Baldwin,41412.0,43004.0,44079.0,4,3.8,2.5,66
2,Alabama,Barbour,31509.0,31750.0,33453.0,46,0.8,5.4,11
3,Alabama,Bibb,29048.0,28989.0,30022.0,66,-0.2,3.6,47
4,Alabama,Blount,31987.0,32261.0,33707.0,45,0.9,4.5,19


In [8]:
income_df.rename(columns={ income_df.columns[2]: "2015($)" }, inplace = True)
income_df.rename(columns={ income_df.columns[3]: "2016($)" }, inplace = True)
income_df.rename(columns={ income_df.columns[4]: "2017($)" }, inplace = True)
income_df.rename(columns = {income_df.columns[0]: "State"}, inplace = True)
income_df.rename(columns = {income_df.columns[1]: "County"}, inplace = True)

income_df.head()

,State,County,2015($),2016($),2017($),Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Alabama,Autauga,38591.0,39509.0,40484.0,9,2.4,2.5,67
1,Alabama,Baldwin,41412.0,43004.0,44079.0,4,3.8,2.5,66
2,Alabama,Barbour,31509.0,31750.0,33453.0,46,0.8,5.4,11
3,Alabama,Bibb,29048.0,28989.0,30022.0,66,-0.2,3.6,47
4,Alabama,Blount,31987.0,32261.0,33707.0,45,0.9,4.5,19


In [9]:
year2015_data = income_df["2015($)"]
year2016_data = income_df["2016($)"]
year2017_data = income_df["2017($)"]

income_state_name=income_df["State"]
income_county=income_df["County"]


In [10]:
rename_income_df= pd.DataFrame({"State": income_state_name,
                        "County": income_county,
#                         "2015($)": year2015_data,
                        "Income Per Capita 2016($)": year2016_data,
#                         "2017($)": year2017_data,
                       })
rename_income_df.head()

,State,County,Income Per Capita 2016($)
0,Alabama,Autauga,39509.0
1,Alabama,Baldwin,43004.0
2,Alabama,Barbour,31750.0
3,Alabama,Bibb,28989.0
4,Alabama,Blount,32261.0


In [23]:
merge_states_info = pd.merge(formatted_df, rename_income_df, 
                             right_on=["State", "County"], left_on = ["State", "County"])
merge_states_info.head()

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%),Income Per Capita 2016($)
0,Alabama,AL,Autauga,5.883047,30.9,56.470750,83.000000,39509.0
1,Alabama,AL,Baldwin,6.053720,26.7,62.335918,80.000000,43004.0
2,Alabama,AL,Barbour,10.809275,40.8,43.358568,80.530303,31750.0
3,Alabama,AL,Bibb,7.137850,40.1,48.927875,81.000000,28989.0
4,Alabama,AL,Blount,6.146908,32.4,49.149934,84.975362,32261.0


In [28]:
reduced_SNAP_df = SNAP_df.loc[:, ["State Abbr", "County", "July 2016"]]
reduced_SNAP_df.rename(columns={"July 2016": "Number of SNAP Recipents"}, inplace=True)
reduced_SNAP_df = reduced_SNAP_df[reduced_SNAP_df["State Abbr"].astype(str) != "None"]

reduced_SNAP_df.head()

,State Abbr,County,Number of SNAP Recipents
1,AL,Autauga,7844.0
2,AL,Baldwin,21406.0
3,AL,Barbour,6675.0
4,AL,Bibb,3740.0
5,AL,Blount,7037.0


In [29]:
reduced_SNAP_df['State Abbr'] = reduced_SNAP_df['State Abbr'].str.strip()
reduced_SNAP_df['County'] = reduced_SNAP_df['County'].str.strip()
merge_states_info['State Abbr'] = merge_states_info['State Abbr'].str.strip()
merge_states_info['County'] = merge_states_info['County'].str.strip()
all_merged_df = pd.merge(merge_states_info, reduced_SNAP_df, 
                         right_on = ["State Abbr","County"], left_on = ["State Abbr","County"])
all_merged_df.head()
# for x in merge_states_info['County'].astype('str'):
#      print(f'"{x}"')

# merge_states_info.dtypes
# merge_states_info.loc[[0,1,2],:]


,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%),Income Per Capita 2016($),Number of SNAP Recipents
0,Alabama,AL,Autauga,5.883047,30.9,56.470750,83.000000,39509.0,7844.0
1,Alabama,AL,Baldwin,6.053720,26.7,62.335918,80.000000,43004.0,21406.0
2,Alabama,AL,Barbour,10.809275,40.8,43.358568,80.530303,31750.0,6675.0
3,Alabama,AL,Bibb,7.137850,40.1,48.927875,81.000000,28989.0,3740.0
4,Alabama,AL,Blount,6.146908,32.4,49.149934,84.975362,32261.0,7037.0


In [35]:
all_merged_df.count()

State                        2961
State Abbr                   2961
County                       2961
Unemployment (%)             2961
Obesity (%)                  2961
Some College (%)             2961
High School Graduate (%)     2479
Income Per Capita 2016($)    2961
Number of SNAP Recipents     2961
dtype: int64

In [40]:
#drop rows missing data
all_merged_df.dropna(inplace=True)

In [41]:
all_merged_df.count()

State                        2479
State Abbr                   2479
County                       2479
Unemployment (%)             2479
Obesity (%)                  2479
Some College (%)             2479
High School Graduate (%)     2479
Income Per Capita 2016($)    2479
Number of SNAP Recipents     2479
dtype: int64

In [42]:
#Get sample of 30% of population
sample = all_merged_df.sample(frac =.30)
sample.head()

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%),Income Per Capita 2016($),Number of SNAP Recipents
841,Kansas,KS,Bourbon,5.218003,36.1,61.433447,83.073980,39031.0,1834.0
1715,New York,NY,Chemung,6.264966,31.9,58.983623,73.598176,40520.0,15163.0
2655,Utah,UT,Beaver,3.942049,27.6,57.836788,82.500000,28301.0,481.0
2835,West Virginia,WV,Marion,5.927436,34.4,57.751938,83.000000,38092.0,9633.0
655,Indiana,IN,Carroll,5.276731,28.6,50.822155,90.495283,40566.0,1302.0


In [43]:
sample.count()

State                        744
State Abbr                   744
County                       744
Unemployment (%)             744
Obesity (%)                  744
Some College (%)             744
High School Graduate (%)     744
Income Per Capita 2016($)    744
Number of SNAP Recipents     744
dtype: int64